In [ ]:
# load the LSST stack
# source ~/lsst_devel/LSST/lsstsw/bin/setup.sh 
# setup pipe_tasks -t b1843
# setup -j -r ~/lsst_devel/LSST/repos/ticket3704/obs_decam/

In [ ]:
import lsst.pipe.base as pipeBase
from lsst.pipe.tasks.processCcd import ProcessCcdTask
from lsst.obs.decam.decamNullIsr import DecamNullIsrTask
from lsst.ip.isr import IsrTask
import lsst.pex.config as pexConfig
import lsst.pipe.base as pipeBase
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig 

In [ ]:
# try out one exposure
visit=288935
ccdnum = 22
filterName='g'

In [ ]:
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler('/Users/yusra/calexp_dir_Blind14A_10')
# read in exposure
# ccd level keys are visit, ccdnum and filter. filter for this dataset is 'g'
# IF you want them to look nice: use github.com:yalsayyad/obs_decam (look out for branch number that reads in 
#        crblasted files instead of raw files)

exposure = butler.get("instcal", visit=visit, ccdnum=ccdnum, filter='g', immediate=True)
exposureIdInfo = butler.get("expIdInfo", visit=visit, ccdnum=ccdnum, filter='g', immediate=True)
exposure.writeFits('filename.fits')


If you want the butler to read in the smaller instcals (per ccd) in the crblasted files, use github.com/yalsayyad/obs_decam


To viz a whole visit in ds9

$ ds9 -mosaic wcs *

In [ ]:
# Characterize Image
charImageConfig = CharacterizeImageConfig()
charImage = CharacterizeImageTask()
# charImage.characterize(exposure, exposureIdInfo=exposureIdInfo,  background=None) 
# OR
charRes = charImage.characterize(exposure) #Everything else CAN be None
exposure = charRes.exposure
bkgd = charRes.background

# Calibrate Image
calibrateConfig = CalibrateConfig(doPhotoCal=False, doAstrometry=False)
calibrateTask = CalibrateTask(config=calibrateConfig)
calibRes = calibrateTask.calibrate(exposure, 
                        exposureIdInfo=None, background=bkgd, icSourceCat=None)

# In original https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py
# the run method is called like:             calibRes = self.calibrate.run(
#                dataRef = sensorRef,
#                exposure = charRes.exposure,
#                background = charRes.background,
#                doUnpersist = False,
#                icSourceCat = charRes.sourceCat,
#            )

#Set results from processCCD, these are going to be the inputs to makeCoaddTempExp
charRes = charRes
calibRes = calibRes if self.config.doCalibrate else None
exposure = exposure
background = calibRes.background if self.config.doCalibrate else charRes.background



In [ ]:
# to find out which patch overlaps the one ccd we processed 
# ("exposure" now contains the "calexp")
wcs = exposure.getWcs()
tract = hits_skymap[0]
tract.findPatch(wcs.pixelToSky(1000,1000))

# answer: PatchInfo(index=(16, 17), innerBBox=Box2I((32000, 34000), (33999, 35999)), outerBBox=Box2I((31900, 33900), (34099, 36099)))


In [ ]:
def getSkyInfo(skyMap, xIndex, yIndex, tractId=0):
    tractInfo = skyMap[tractId]
    # patch format is "xIndex,yIndex"
    patchInfo = tractInfo.getPatchInfo((xIndex, yIndex))
    return pipeBase.Struct(
        skyMap = skyMap,
        tractInfo = tractInfo,
        patchInfo = patchInfo,
        wcs = tractInfo.getWcs(),
        bbox = patchInfo.getOuterBBox(),
    )

skyInfo = getSkyInfo(hits_skymap, 16, 17, tractId=0)

In [ ]:
# For each patch: 
#     For each visit that overlaps the patch (assume all):
#        makeCoaddTempExp for every visit by doing...
#        For each ccd in that visit that overlaps the patch:
#            warpAndPsfMatch.run
#            coaddUtils.copyGoodPixels  

In [ ]:
# for this part we are going to assume that we are deep in the loop:
xIndex, yIndex = (16, 17)
#filterName='g'
# visit=289853
#ccdnum = 22

In [ ]:
from lsst.pipe.tasks.makeCoaddTempExp import MakeCoaddTempExpTask, MakeCoaddTempExpConfig
import lsst.afw.image as afwImage
import lsst.coadd.utils as coaddUtils
from lsst.meas.algorithms import CoaddPsf
import numpy
makeCTEConfig = MakeCoaddTempExpConfig()
makeCTE = MakeCoaddTempExpTask(config=makeCTEConfig)

In [ ]:
coaddTempExp = afwImage.ExposureF(skyInfo.bbox, skyInfo.wcs)
coaddTempExp.getMaskedImage().set(numpy.nan, afwImage.MaskU.getPlaneBitMask("NO_DATA"), numpy.inf)
totGoodPix = 0
didSetMetadata = False
modelPsf = makeCTEConfig.modelPsf.apply() if makeCTEConfig.doPsfMatch else None
# number of 
ccdsInVisit = 1
inputRecorder =  makeCTE.inputRecorder.makeCoaddTempExpRecorder(visit, ccdsInVisit)
ccdId = exposure.getId()


### NOW you start the loop over the 1-4 ccd's IN THIS VISIT that overlap this patch
numGoodPix = 0
calExp = exposure
# We MIGHT need to subtract the backgrounds here. Double check.
warpedCcdExp = makeCTE.warpAndPsfMatch.run(calExp, modelPsf=modelPsf, 
                                           wcs=skyInfo.wcs,maxBBox=skyInfo.bbox).exposure
if didSetMetadata:
    mimg = exposure.getMaskedImage()
    mimg *= (coaddTempExp.getCalib().getFluxMag0()[0] / calExp.getCalib().getFluxMag0()[0])
    del mimg

numGoodPix = coaddUtils.copyGoodPixels(coaddTempExp.getMaskedImage(),
                                       warpedCcdExp.getMaskedImage(),
                                       makeCTE.getBadPixelMask())
totGoodPix += numGoodPix
if numGoodPix > 0 and not didSetMetadata:
    coaddTempExp.setCalib(warpedCcdExp.getCalib())
    coaddTempExp.setFilter(warpedCcdExp.getFilter())
    didSetMetadata = True

#since we are not looping do this again. But this should only be in the loop once at the beginngin

if didSetMetadata:
    mimg = exposure.getMaskedImage()
    mimg *= (coaddTempExp.getCalib().getFluxMag0()[0] / exposure.getCalib().getFluxMag0()[0])
    del mimg

inputRecorder.addCalExp(calExp, ccdId, numGoodPix)

##### End loop over ccds here:
inputRecorder.finish(coaddTempExp, totGoodPix)

if totGoodPix > 0 and didSetMetadata:
    coaddTempExp.setPsf(modelPsf if makeCTEConfig.doPsfMatch else
                    CoaddPsf(inputRecorder.coaddInputs.ccds, skyInfo.wcs))
    

In [ ]:
coaddTempDict = {}
coaddTempDict[visit] = coaddTempExp

In [ ]:
from lsst.pipe.tasks.assembleCoadd import AssembleCoaddTask, AssembleCoaddConfig
import lsst.afw.math as afwMath
config = AssembleCoaddConfig()
assembleTask = AssembleCoaddTask(config=config)

def getTempExpDatasetName():
    return 'Deep'


def prepareInputs(cteList, dataIdList, assembleTask):
    """!
    \brief Prepare the input warps for coaddition by measuring the weight for each warp and the scaling 
    for the photometric zero point.

    Each coaddTempExp has its own photometric zeropoint and background variance. Before coadding these 
    coaddTempExps together, compute a scale factor to normalize the photometric zeropoint and compute the
    weight for each coaddTempExp. 

    \param[in] refList: List of data references to tempExp
    \return Struct:
    - tempExprefList: List of data references to tempExp
    - weightList: List of weightings
    - imageScalerList: List of image scalers
    """
    statsCtrl = afwMath.StatisticsControl()
    statsCtrl.setNumSigmaClip(assembleTask.config.sigmaClip)
    statsCtrl.setNumIter(assembleTask.config.clipIter)
    statsCtrl.setAndMask(assembleTask.getBadPixelMask())
    statsCtrl.setNanSafe(True)
    # compute tempExpRefList: a list of tempExpRef that actually exist
    # and weightList: a list of the weight of the associated coadd tempExp
    # and imageScalerList: a list of scale factors for the associated coadd tempExp
    newDataIdList = [] #make clean list incase scaling failed. output lists should all be same length
    weightList = []
    imageScalerList = []
    tempExpName = getTempExpDatasetName()
    for dataId, tempExp in zip(dataIdList, cteList):
        maskedImage = tempExp.getMaskedImage()
        imageScaler = assembleTask.scaleZeroPoint.computeImageScaler(
            exposure = tempExp,
            dataRef = None,
        )
        try:
            pass
            # You can comment this out
            #imageScaler.scaleMaskedImage(maskedImage) #warning. This changes the images!
        except Exception as e:
            print("Scaling failed for %s (skipping it): %s" % (tempExpRef.dataId, e))
            continue
        statObj = afwMath.makeStatistics(maskedImage.getVariance(), maskedImage.getMask(),
            afwMath.MEANCLIP, statsCtrl)
        meanVar, meanVarErr = statObj.getResult(afwMath.MEANCLIP);
        weight = 1.0 / float(meanVar)
        if not numpy.isfinite(weight):
            print("Non-finite weight for %s: skipping" % (dataId))
            continue
        print("Weight of %s %s = %0.3f" % (tempExpName, dataId, weight))
        del maskedImage
        del tempExp
        newDataIdList.append(dataId)
        weightList.append(weight)
        imageScalerList.append(imageScaler)
    return pipeBase.Struct(dataIdList=newDataIdList, weightList=weightList,
                           imageScalerList=imageScalerList)

In [ ]:
imageScalerRes = prepareInputs(coaddTempDict.values(), coaddTempDict.keys(), assembleTask)

mask = None
doClip=False
if mask is None:
    mask = assembleTask.getBadPixelMask()

statsCtrl = afwMath.StatisticsControl()
statsCtrl.setNumSigmaClip(assembleTask.config.sigmaClip)
statsCtrl.setNumIter(assembleTask.config.clipIter)
statsCtrl.setAndMask(mask)
statsCtrl.setNanSafe(True)
statsCtrl.setWeighted(True)
statsCtrl.setCalcErrorFromInputVariance(True)
for plane, threshold in assembleTask.config.maskPropagationThresholds.items():
    bit = afwImage.MaskU.getMaskPlane(plane)
    statsCtrl.setMaskPropagationThreshold(bit, threshold)

if doClip:
    statsFlags = afwMath.MEANCLIP
else:
    statsFlags = afwMath.MEAN

coaddExposure = afwImage.ExposureF(skyInfo.bbox, skyInfo.wcs)
coaddExposure.setCalib(assembleTask.scaleZeroPoint.getCalib())
coaddExposure.getInfo().setCoaddInputs(assembleTask.inputRecorder.makeCoaddInputs())

#remember to set metadata if you want any hope of running detection and measurement on this coadd:
#self.assembleMetadata(coaddExposure, tempExpRefList, weightList)

#most important thing is the psf
coaddExposure.setFilter(coaddTempDict.values()[0].getFilter())
coaddInputs = coaddExposure.getInfo().getCoaddInputs()

for tempExp, weight in zip(coaddTempDict.values(), imageScalerRes.weightList):
    assembleTask.inputRecorder.addVisitToCoadd(coaddInputs, tempExp, weight)

#takes numCcds as argument
import lsst.meas.algorithms as measAlg
coaddInputs.ccds.reserve(1)
coaddInputs.visits.reserve(len(imageScalerRes.dataIdList))
psf = measAlg.CoaddPsf(coaddInputs.ccds, coaddExposure.getWcs())
coaddExposure.setPsf(psf)

In [ ]:
maskedImageList = afwImage.vectorMaskedImageF()
coaddMaskedImage = coaddExposure.getMaskedImage()

for dataId, imageScaler, exposure in zip(imageScalerRes.dataIdList,
                                         imageScalerRes.imageScalerList, 
                                         coaddTempDict.values()):
    print dataId, imageScaler, exposure
    maskedImage = exposure.getMaskedImage()
    imageScaler.scaleMaskedImage(maskedImage)
    maskedImageList.append(maskedImage)



maskedImage = afwMath.statisticsStack(maskedImageList, statsFlags, statsCtrl, imageScalerRes.weightList)

coaddMaskedImage.assign(maskedImage, skyInfo.bbox)
coaddUtils.setCoaddEdgeBits(coaddMaskedImage.getMask(),
                            coaddMaskedImage.getVariance())


#write out coadd!
coaddExposure.writeFits('coaddOutputExample.fits')

In [ ]:
from lsst.pipe.tasks.multiBand import DetectCoaddSourcesTask, DetectCoaddSourcesConfig 
config = DetectCoaddSourcesConfig()
detectCoaddSources = DetectCoaddSourcesTask(config=config)
detRes = detectCoaddSources.runDetection(coaddExposure, idFactory=None)

In [ ]:
# EXAMPLE Fake Butler DataRef
# If we loop through these objects instead of plain dictionaries,
# Then we can use the original LSST pipe tasks

class quickCalexpRef(object):
    def init(self, visit, ccd):
        self.dataId = dict(visit=visit, ccd=ccd, filterName='g')
    def get(self, dataType, **kwargs):
        if dataType == 'instcal':
            print "I'm looking on disk or DB for: %s_%s_%s"%('instcal', 
                                                             self.dataId['visit'],
                                                             self.dataId['ccd'])  
        if dataType == 'calexp':
            print "I'm looking on disk or DB for: %s_%s_%s"%('calexp',
                                                             self.dataId['visit'],
                                                             self.dataId['ccd'])
class quickTempExpRef(object):
    def __init__(self, tract, patch, visit, filterName):
        self.dataId = dict(tract=tract, patch=patch, visit=visit)
    def get(self, dataType, **kwargs):
        if dataType == 'CoaddTempExp':
            print "I'm looking on disk or DB for: %s_%s_%s_%s"%('CoaddTempExp', 
                                                             self.dataId['tract'],
                                                             self.dataId['patch'],
                                                             self.dataId['visit'])  

# More Automated

In [ ]:
# a little more automated:
# PROCESS CCD
visitList = [289893, 289820, 288935]
ccdList = [22, 28, 29, 16]
exposureDict = {}
calexpDict = {}
for visit in visitList:
    exposureDict[visit] = {}
    calexpDict[visit] = {}
    for ccd in ccdList:
        exposureDict[visit][ccd] = butler.get("instcal", visit=visit, 
                                              ccdnum=ccd, filter='g', immediate=True)
        try:
            charRes = charImage.characterize(exposureDict[visit][ccd]) #Everything else CAN be None
            calibRes = calibrateTask.calibrate(charRes.exposure, 
                        exposureIdInfo=None, background=charRes.background,
                                           icSourceCat=None)
        except:
            print ccd, ' ', visit, ' failed. '
            continue
        calexpDict[visit][ccd] = calibRes


# MAKE COADD TEMP EXP
patch = (xIndex, yIndex)
coaddTempExpDict = {}
coaddTempExpDict[patch] = {}
for visit in visitList:
    coaddTempExp = afwImage.ExposureF(skyInfo.bbox, skyInfo.wcs)
    coaddTempExp.getMaskedImage().set(numpy.nan, afwImage.MaskU.getPlaneBitMask("NO_DATA"), numpy.inf)
    totGoodPix = 0
    didSetMetadata = False
    modelPsf = makeCTEConfig.modelPsf.apply() if makeCTEConfig.doPsfMatch else None
    # number of 
    ccdsInVisit = 3
    inputRecorder =  makeCTE.inputRecorder.makeCoaddTempExpRecorder(visit, ccdsInVisit)
    for ccd in ccdList:
        ### NOW you start the loop over the 1-4 ccd's IN THIS VISIT that overlap this patch
        numGoodPix = 0
        try:
            calExp = calexpDict[visit][ccd].exposure
        except:
            continue
        ccdId = calExp.getId()
        if didSetMetadata: #then scale image to zeropoint of first ccd
            mimg = calExp.getMaskedImage()
            mimg *= (coaddTempExp.getCalib().getFluxMag0()[0] / calExp.getCalib().getFluxMag0()[0])
            del mimg
        # We MIGHT need to subtract the backgrounds here. Double check.
        warpedCcdExp = makeCTE.warpAndPsfMatch.run(calExp, modelPsf=modelPsf, 
                                                   wcs=skyInfo.wcs,maxBBox=skyInfo.bbox).exposure
        numGoodPix = coaddUtils.copyGoodPixels(coaddTempExp.getMaskedImage(),
                                               warpedCcdExp.getMaskedImage(),
                                               makeCTE.getBadPixelMask())
        totGoodPix += numGoodPix
        print ccd, numGoodPix
        if numGoodPix > 0 and not didSetMetadata:
            coaddTempExp.setCalib(warpedCcdExp.getCalib())
            coaddTempExp.setFilter(warpedCcdExp.getFilter())
            didSetMetadata = True
        inputRecorder.addCalExp(calExp, ccdId, numGoodPix)
        ##### End loop over ccds here:
    inputRecorder.finish(coaddTempExp, totGoodPix)
    if totGoodPix > 0 and didSetMetadata:
        coaddTempExp.setPsf(modelPsf if makeCTEConfig.doPsfMatch else
            CoaddPsf(inputRecorder.coaddInputs.ccds, skyInfo.wcs))
    coaddTempExpDict[patch][visit] = coaddTempExp

#coaddTempExpDict[patch][visit].writeFits('%s.fits'%(visit))


# Now assemble coadd per patch
imageScalerRes = prepareInputs(coaddTempExpDict[patch].values(),
                               coaddTempExpDict[patch].keys(),
                               assembleTask)

mask = None
doClip=False
if mask is None:
    mask = assembleTask.getBadPixelMask()

statsCtrl = afwMath.StatisticsControl()
statsCtrl.setNumSigmaClip(assembleTask.config.sigmaClip)
statsCtrl.setNumIter(assembleTask.config.clipIter)
statsCtrl.setAndMask(mask)
statsCtrl.setNanSafe(True)
statsCtrl.setWeighted(True)
statsCtrl.setCalcErrorFromInputVariance(True)
for plane, threshold in assembleTask.config.maskPropagationThresholds.items():
    bit = afwImage.MaskU.getMaskPlane(plane)
    statsCtrl.setMaskPropagationThreshold(bit, threshold)

if doClip:
    statsFlags = afwMath.MEANCLIP
else:
    statsFlags = afwMath.MEAN

coaddExposure = afwImage.ExposureF(skyInfo.bbox, skyInfo.wcs)
coaddExposure.setCalib(assembleTask.scaleZeroPoint.getCalib())
coaddExposure.getInfo().setCoaddInputs(assembleTask.inputRecorder.makeCoaddInputs())


#remember to set metadata if you want any hope of running detection and measurement on this coadd:
#self.assembleMetadata(coaddExposure, tempExpRefList, weightList)

#most important thing is the psf
coaddExposure.setFilter(coaddTempExpDict[patch].values()[0].getFilter())
coaddInputs = coaddExposure.getInfo().getCoaddInputs()

for tempExp, weight in zip(coaddTempExpDict[patch].values(), imageScalerRes.weightList):
    assembleTask.inputRecorder.addVisitToCoadd(coaddInputs, tempExp, weight)

#takes numCcds as argument
import lsst.meas.algorithms as measAlg
coaddInputs.ccds.reserve(3)
coaddInputs.visits.reserve(len(imageScalerRes.dataIdList))
psf = measAlg.CoaddPsf(coaddInputs.ccds, coaddExposure.getWcs())
coaddExposure.setPsf(psf)


maskedImageList = afwImage.vectorMaskedImageF()
coaddMaskedImage = coaddExposure.getMaskedImage()
for dataId, imageScaler, exposure in zip(imageScalerRes.dataIdList,
                                         imageScalerRes.imageScalerList, 
                                         coaddTempExpDict[patch].values()):
    print dataId, imageScaler, exposure
    maskedImage = exposure.getMaskedImage()
    imageScaler.scaleMaskedImage(maskedImage)
    maskedImageList.append(maskedImage)


maskedImage = afwMath.statisticsStack(maskedImageList, 
                                      statsFlags, statsCtrl,
                                      imageScalerRes.weightList)

coaddMaskedImage.assign(maskedImage, skyInfo.bbox)
coaddUtils.setCoaddEdgeBits(coaddMaskedImage.getMask(),
                            coaddMaskedImage.getVariance())

# write out Coadd!
#coaddExposure.writeFits('16_17.fits')

# Detect on Coadd
config = DetectCoaddSourcesConfig()
detectCoaddSources = DetectCoaddSourcesTask(config=config)
detRes = detectCoaddSources.runDetection(coaddExposure, idFactory=None)


In [ ]:
# Output catalog just has positions. 
detRes.sources

#to put this source catalog into a more familiar format:
import pandas as pd
df=pd.DataFrame()
sourceCat = detRes.sources
for col in sourceCat.getSchema().getNames():
    df[col] = sourceCat[col]

In [ ]:
# Measurement on Coadds not working yet:
#from lsst.pipe.tasks.multiBand import MergeDetectionsTask, MergeDetectionsConfig
#from lsst.pipe.tasks.multiBand import MeasureMergedCoaddSourcesTask, MeasureMergedCoaddSourcesConfig

#config =  MergeDetectionsConfig()
#mergeDetectionsTask = MergeDetectionsTask(config=config)
#config =  MeasureMergedCoaddSourcesConfig()
#measureCoaddSourcesTask = MeasureMergedCoaddSourcesTask(detRes.sources.getSchema())
#measureCoaddSourcesTask.measure(detRes.sources, coaddExposure)

#from lsst.meas.base import SingleFrameMeasurementTask, BasePlugin, ApplyApCorrTask, AfterburnerTask
#schema = afwTable.SourceTable.makeMinimalSchema()
#task = SingleFrameMeasurementTask(schema)
#task.measure(detRes.sources, coaddExposure)